In [16]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re


if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/Desktop/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Methods.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

NameError: name 'path_equity_precision_llm_folder' is not defined

In [14]:
results = pd.read_csv(path_equity_precision_llm_folder + '/llm training/Training_results.csv')
input = pd.read_excel(excel_path, sheet_name='Training Data')


NameError: name 'path_equity_precision_llm_folder' is not defined

In [4]:
# input['orig_source_population'] = input['Source Population'].apply(lambda x: ' '.join([word[0] for word in x.split()])).str.replace(r'\s', '')
input['pubmed_source_population'] = input['Source Population'].apply(lambda x: ''.join([word[0] for word in x.split()])).str.replace(r'&', '')
input = input.rename(columns={'Precision Medicine': 'orig_precision_medicine', 
                                  'Diabetes': 'orig_diabetes', 
                                  'Primary Study': 'orig_primary_study', 
                                  'Correct Source Population': 'binary_source_population'})

# Convert to lower
input['orig_precision_medicine'] = input['orig_precision_medicine'].str.lower()
input['orig_diabetes'] = input['orig_diabetes'].str.lower()
input['orig_primary_study'] = input['orig_primary_study'].str.lower()
input['binary_source_population'] = input['binary_source_population'].str.lower()



input.drop(columns=['Source Population'], inplace=True)
input.head()




,PMID,Title,Abstract,MeSH,orig_precision_medicine,orig_diabetes,binary_source_population,orig_primary_study,pubmed_source_population
0,22744164,Acculturation and glycemic control of Asian In...,The prevalence of type 2 diabetes is dispropor...,Acculturation*; Asian / psychology; Asian / st...,no,yes,yes,yes,SA
1,15561964,Linkage analysis of diabetes status among hype...,Type 2 diabetes susceptibility is determined b...,"Chromosome Mapping*; Diabetes Mellitus, Type 2...",yes,yes,no,yes,SA
2,28770629,"Ipragliflozin, a sodium glucose co-transporter...",Objective: We recently investigated the effect...,"Blood Glucose / analysis; Diabetes Mellitus, T...",no,yes,yes,yes,EA
3,33764184,Gastrodin protects against high glucose-induce...,Diabetic cardiomyopathy (DCM) is one of the ma...,Aryl Hydrocarbon Receptor Nuclear Translocator...,no,yes,no,yes,EA
4,36155119,Curcumin supplementation reduces blood glucose...,Objective: To evaluate the effect of curcumin ...,Blood Glucose* / metabolism; Body Mass Index; ...,no,no,yes,yes,LAC


In [5]:
merged_df = input.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df.head()

,PMID,Title,Abstract,MeSH,orig_precision_medicine,orig_diabetes,binary_source_population,orig_primary_study,pubmed_source_population,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population
0,22744164,Acculturation and glycemic control of Asian In...,The prevalence of type 2 diabetes is dispropor...,Acculturation*; Asian / psychology; Asian / st...,no,yes,yes,yes,SA,22744164,Acculturation and glycemic control of Asian I...,no,yes,yes,SA
1,15561964,Linkage analysis of diabetes status among hype...,Type 2 diabetes susceptibility is determined b...,"Chromosome Mapping*; Diabetes Mellitus, Type 2...",yes,yes,no,yes,SA,15561964,Linkage analysis of diabetes status among hyp...,yes,yes,yes,NA
2,28770629,"Ipragliflozin, a sodium glucose co-transporter...",Objective: We recently investigated the effect...,"Blood Glucose / analysis; Diabetes Mellitus, T...",no,yes,yes,yes,EA,28770629,"Ipragliflozin, a sodium glucose co-transporte...",no,yes,yes,EA
3,33764184,Gastrodin protects against high glucose-induce...,Diabetic cardiomyopathy (DCM) is one of the ma...,Aryl Hydrocarbon Receptor Nuclear Translocator...,no,yes,no,yes,EA,33764184,Gastrodin protects against high glucose-induc...,yes,yes,yes,UNK
4,36155119,Curcumin supplementation reduces blood glucose...,Objective: To evaluate the effect of curcumin ...,Blood Glucose* / metabolism; Body Mass Index; ...,no,no,yes,yes,LAC,36155119,Curcumin supplementation reduces blood glucos...,no,yes,yes,LAC


In [6]:
pd.crosstab(merged_df['gpt_precision_medicine'], merged_df['orig_precision_medicine'])


orig_precision_medicine,no,yes
gpt_precision_medicine,,
no,9,2
yes,1,8


In [7]:
pd.crosstab(merged_df['gpt_diabetes'], merged_df['orig_diabetes'])


orig_diabetes,no,yes
gpt_diabetes,,
no,4,0
yes,5,11


In [8]:
pd.crosstab(merged_df['gpt_primary_study'], merged_df['orig_primary_study'])


orig_primary_study,yes
gpt_primary_study,
yes,20
